In [6]:
# Generall imports
import sys
from pathlib import Path

# Ensure project root (OMNIROUTE_ANALYSIS/) is on sys.path
project_root = Path().resolve().parents[1]
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# project imports
from utils.config import Session
import utils.io_trodes as io_trodes
import utils.io_rosbag as io_rosbag
import utils.io_ts_sync as io_ts_sync

# Parameters
session_folder = r"C:\Users\lester\UBC\Madhav, Manu - lesterkaur2024gate\analysis\gate_ephys_test\NC40008\20250328_134136"

sync_channel = 1  # SpikeGadgets DIO channel used for sync pulses (Din01)

# Initialize session object
sess = Session(session_folder)

# Load digital sync timestamps from SpikeGadgets
sg_sync_ts = io_trodes.load_dio_events(sess, channel=sync_channel)
if len(sg_sync_ts) == 0:
    raise RuntimeError("No sync pulses found in SpikeGadgets DIO data.")

# Load digital sync timestamps from ROS bag
ros_sync_ts = io_rosbag.load_ros_sync_events(sess)
if len(ros_sync_ts) == 0:
    raise RuntimeError("No sync pulses found in ROS bag.")

# Compute timestamp alignment (SG → ROS) using polynomial regression
sync_fit_coeffs = io_ts_sync.align_timestamps(sg_sync_ts, ros_sync_ts)

# Save the computed mapping
io_ts_sync.save_sync_fit(sess, sync_fit_coeffs)

print(f"Sync parameters saved to: {sess.get_sync_fit_path()}")


FileNotFoundError: DIO file not found: C:\Users\lester\UBC\Madhav, Manu - lesterkaur2024gate\analysis\gate_ephys_test\NC40008\20250328_134136\extracted\20250328_merged.DIO\Din01.dat